In [1]:
import numpy as np
import scipy.sparse as sparse
import scipy
import scipy.optimize as opt
import matplotlib.pyplot as plt
import generate_constant_matrices as mat_gen

In [2]:
N_omega = 101
N_z = 21
omega = np.linspace(-3, 3, N_omega)
N_proj = 21

In [13]:
list_proj, list_proj_conj = mat_gen.get_lin_proj_mat(N_z, N_omega, N_proj, real=False)

In [19]:
sparse.csr_matrix((1, 10)).toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])